In [1]:
import os

def move_to_parent_dir():
    current_directory = os.getcwd()
    parent_directory = os.path.dirname(current_directory)
    os.chdir(parent_directory)
    print("Working directory:", os.getcwd())

move_to_parent_dir()

Working directory: c:\Users\wwden\OneDrive\Documents\GitHub\dsmp-2024-group-19


In [2]:
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
import numpy as np
from tools import get_LOBs

for day in range(125):
    df = get_LOBs(1,day)[0]

    date = str(df.index[0].date())

    print(f"LOB: {date}")
    N = len(df)
    LOB_arr = np.zeros((N,800), dtype = np.int8)
    row_i = 0
    for i, row in df.iterrows():

        bids = row["LOB"][0][1]
        asks = row["LOB"][1][1]

        for price, size in bids:
            LOB_arr[row_i, price - 1] = -size

        for price, size in asks:
            LOB_arr[row_i, price - 1] = size

        row_i += 1



    print("Density:", 1 - np.mean(LOB_arr == 0))
    save_npz(os.path.join("CSR_Data", f"CSR_LOB_{date}.npz"), csr_matrix(LOB_arr, dtype=np.int8))
    with open(os.path.join("CSR_Data", f"Timestamps_{date}.csv"), 'w') as f:
        for timestamp in df.index:
            f.write(str(timestamp.time()) + "\n")


Opening UoB_Set01_2025-01-02LOBs.txt
LOB: 2025-01-02
Density: 0.018302037000311633
Opening UoB_Set01_2025-01-03LOBs.txt
LOB: 2025-01-03
Density: 0.016811435298289212
Opening UoB_Set01_2025-01-06LOBs.txt
LOB: 2025-01-06
Density: 0.018428771405774813
Opening UoB_Set01_2025-01-07LOBs.txt
LOB: 2025-01-07
Density: 0.019879855302015637
Opening UoB_Set01_2025-01-08LOBs.txt
LOB: 2025-01-08


In [ ]:
from scipy.sparse import load_npz

# Load the sparse matrix from the "test.npz" file
loaded_array = load_npz("test.npz")

# Convert the loaded sparse matrix to a dense NumPy array
dense_array = loaded_array.toarray()

dense_array

In [ ]:
def CBS(c):
    col = c.copy()
    col[col >= 0] = 0
    diff = np.abs(np.diff(col))
    return np.sum(diff) + col[0]

def CAS(c):
    col = c.copy()
    col[col <= 0] = 0
    diff = np.abs(np.diff(col))
    return np.sum(diff) + col[0]

cbs = np.apply_along_axis(CBS, axis=0, arr=dense_array)
cas = np.apply_along_axis(CAS, axis=0, arr=dense_array)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))
plt.plot(np.arange(1,801),cbs, label="cbs")
plt.plot(np.arange(1,801),cas, label="cas")
plt.legend()

In [ ]:
CAS(dense_array[:,799])

In [ ]:
dense_array[:,799]